# PyTorch Assignment: Multi-Layer Perceptron (MLP)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: 

### Multi-Layer Perceptrons

The simple logistic regression example we went over in the previous notebook is essentially a one-layer neural network, projecting straight from the input to the output predictions.
While this can be effective for linearly separable data, occasionally a little more complexity is necessary.
Neural networks with additional layers are typically able to learn more complex functions, leading to better performance.
These additional layers (called "hidden" layers) transform the input into one or more intermediate representations before making a final prediction.

In the logistic regression example, the way we performed the transformation was with a fully-connected layer, which consisted of a linear transform (matrix multiply plus a bias).
A neural network consisting of multiple successive fully-connected layers is commonly called a Multi-Layer Perceptron (MLP). 
In the simple MLP below, a 4-d input is projected to a 5-d hidden representation, which is then projected to a single output that is used to make the final prediction.

<img src="Figures/MLP.png" width="300"/>

For the assignment, you will be building a MLP for MNIST.
Mechanically, this is done very similary to our logistic regression example, but instead of going straight to a 10-d vector representing our output predictions, we might first transform to a 500-d vector with a "hidden" layer, then to the output of dimension 10.
Before you do so, however, there's one more important thing to consider.

### Nonlinearities

We typically include nonlinearities between layers of a neural network.
There's a number of reasons to do so.
For one, without anything nonlinear between them, successive linear transforms (fully connected layers) collapse into a single linear transform, which means the model isn't any more expressive than a single layer.
On the other hand, intermediate nonlinearities prevent this collapse, allowing neural networks to approximate more complex functions.

There are a number of nonlinearities commonly used in neural networks, but one of the most popular is the [rectified linear unit (ReLU)](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)):

\begin{align}
x = \max(0,x)
\end{align}

There are a number of ways to implement this in PyTorch.
We could do it with elementary PyTorch operations:

In [1]:
import torch

x = torch.rand(5, 3)*2 - 1
x_relu_max = torch.max(torch.zeros_like(x),x)

print("x: {}".format(x))
print("x after ReLU with max: {}".format(x_relu_max))

x: tensor([[ 0.9123,  0.0382,  0.7757],
        [-0.6771,  0.9026,  0.2795],
        [-0.9797, -0.8080,  0.3947],
        [ 0.0912, -0.5694, -0.5012],
        [-0.1517, -0.0644, -0.7285]])
x after ReLU with max: tensor([[0.9123, 0.0382, 0.7757],
        [0.0000, 0.9026, 0.2795],
        [0.0000, 0.0000, 0.3947],
        [0.0912, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000]])


Of course, PyTorch also has the ReLU implemented, for example in `torch.nn.functional`:

In [2]:
import torch.nn.functional as F

x_relu_F = F.relu(x)

print("x after ReLU with nn.functional: {}".format(x_relu_F))

x after ReLU with nn.functional: tensor([[0.9123, 0.0382, 0.7757],
        [0.0000, 0.9026, 0.2795],
        [0.0000, 0.0000, 0.3947],
        [0.0912, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000]])


Same result.

### Assignment

Build a 2-layer MLP for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image (784 dimensions) ->  
fully connected layer (500 hidden units) -> nonlinearity (ReLU) ->  
fully connected (10 hidden units) -> softmax

Try building the model both with basic PyTorch operations, and then again with more object-oriented higher-level APIs. 
You should get similar results!


*Some hints*:
- Even as we add additional layers, we still only require a single optimizer to learn the parameters.
Just make sure to pass all parameters to it!
- As you'll calculate in the Short Answer, this MLP model has many more parameters than the logisitic regression example, which makes it more challenging to learn.
To get the best performance, you may want to play with the learning rate and increase the number of training epochs.
- Be careful using `torch.nn.CrossEntropyLoss()`. 
If you look at the [PyTorch documentation](https://pytorch.org/docs/stable/nn.html#crossentropyloss): you'll see that `torch.nn.CrossEntropyLoss()` combines the softmax operation with the cross-entropy.
This means you need to pass in the logits (predictions pre-softmax) to this loss.
Computing the softmax separately and feeding the result into `torch.nn.CrossEntropyLoss()` will significantly degrade your model's performance!

In [3]:
### YOUR CODE HERE


import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from torchvision import datasets,transforms

trainset = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
testset = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 10, shuffle = False)
testloader = torch.utils.data.DataLoader(testset, batch_size = 10, shuffle = True)

class MNIST_MLP_model(nn.Module):
  def __init__(self):
    super().__init__()
    #two layers
    #first layer: in_features = 784, out_features = 500, bias = True
    self.fc1 = nn.Linear(28*28,500)
    #second layer: in_features = 500, out_features = 10, bias = True
    self.fc2 = nn.Linear(500,10)
    
#     #three layers and dropout layer
#     self.fc1 = nn.Linear(28 * 28, 512)
#     # linear layer (n_hidden -> hidden_2)
#     self.fc2 = nn.Linear(512, 512)
#     # linear layer (n_hidden -> 10)
#     self.fc3 = nn.Linear(512, 10)
#     # dropout layer (p=0.2) to prevent overfitting
#     self.dropout = nn.Dropout(0.2)
    
  def forward(self, x):
    # flatten image input
    x = x.view(-1, 28 * 28)
    # add hidden layer, with relu activation function
    x = F.relu(self.fc1(x))
    return x

batches = iter(trainloader)

#create model
mymodel = MNIST_MLP_model()

#specify optimizer
optimizer = torch.optim.SGD(mymodel.parameters(),lr=0.01)

#specify loss function
Loss = nn.CrossEntropyLoss()

for batch in tqdm(batches):
    images,labels = batch
    
    #zero out gradients
    optimizer.zero_grad()
    
    #forward pass
    Y = mymodel.forward(images.view(-1,28*28))
    
    #calculate loss
    loss = Loss(Y,labels)
    
    #backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    
    #update parameter step
    optimizer.step()

test_batch = iter(testloader)
correct = 0
total = 0

for batch in tqdm(test_batch):
  images,labels = batch
  for img,lbl in zip(images,labels):
    Y = mymodel.forward(img.view(-1,28*28))
    label = torch.argmax(Y,dim = 1)
    if(label == lbl): correct+=1
    total+=1

print(f"Accuracy: {(correct/total)*100} %")


# Make sure to print out your accuracy on the test set at the end.

100%|██████████| 1000/1000 [00:23<00:00, 43.10it/s]

Accuracy: 90.16999999999999 %


### Short answer
How many trainable parameters does your model have? 
How does this compare to the logisitic regression example?

In [6]:
for name, param in mymodel.named_parameters():
    if param.requires_grad:
        print(name, param.data)

fc1.weight tensor([[ 2.4900e-02, -3.4197e-02, -9.7600e-03,  ..., -2.5982e-02,
          1.9008e-02, -8.8770e-05],
        [ 3.2999e-02,  5.7396e-03,  3.3336e-02,  ..., -1.4518e-02,
         -3.5606e-02, -1.8053e-02],
        [ 9.2958e-03,  2.0254e-02,  1.1265e-02,  ...,  1.6878e-02,
         -3.1727e-02,  5.4675e-03],
        ...,
        [ 1.1013e-02,  3.1099e-02, -2.6997e-04,  ...,  1.0107e-02,
         -3.1533e-02, -6.9481e-03],
        [-2.0901e-02,  2.7770e-02,  3.9839e-04,  ..., -9.5282e-03,
         -1.0199e-02,  1.8007e-02],
        [-2.8265e-02,  1.8996e-02, -1.2456e-02,  ...,  2.3261e-02,
         -1.7390e-03,  1.2066e-02]])
fc1.bias tensor([ 1.4591e-01,  5.6116e-01,  2.4624e-01,  2.3809e-01,  3.3524e-01,
         5.9677e-01,  2.8187e-01,  4.7774e-01, -5.9270e-02,  2.6574e-01,
        -9.4117e-03, -4.0620e-03,  2.6411e-02,  6.7059e-03, -2.6981e-02,
        -3.6593e-02, -3.4353e-02, -5.2203e-04, -2.9561e-02, -2.9720e-02,
        -1.4711e-02, -3.1881e-02, -2.7585e-02, -2.4638e-